In [6]:
import pandas as pd
import numpy as np
from spektral.data import Dataset, Graph
import pickle
from importlib import reload
import dataset
reload(dataset)

pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None) # default='warn'

In [7]:
# Load combined data
df_raw = pd.read_csv('../data/combined.csv')
df = df_raw.copy()

# Load hero feature data
df_features = pd.read_csv('../data/features.csv')
df_features = df_features.set_index('hero_id')

In [195]:
# Generate DotaV1 dataset for 50000 matches at a time
count = 0
total = len(df)
step = 50000

for i in range(0,int(np.ceil(total/step))):
    start = i*step
    end = start+step if (start+step)<total else total
    df_current = df.iloc[start:end]
    print(f'Start: {start} End: {end-1}')
    graphs_current = dataset.DotaV1(df_current, df_features)
    
    filehandler = open(f'../data/dota_datasets/graphs_v1_{start}-{end-1}.pkl','wb')
    pickle.dump(graphs_current, filehandler)

Start: 0 End: 49999
Graph radiant 1000
Graph radiant 2000
Graph radiant 3000
Graph radiant 4000
Graph radiant 5000
Graph radiant 6000
Graph radiant 7000
Graph radiant 8000
Graph radiant 9000
Graph radiant 10000
Graph radiant 11000
Graph radiant 12000
Graph radiant 13000
Graph radiant 14000
Graph radiant 15000
Graph radiant 16000
Graph radiant 17000
Graph radiant 18000
Graph radiant 19000
Graph radiant 20000
Graph radiant 21000
Graph radiant 22000
Graph radiant 23000
Graph radiant 24000
Graph radiant 25000
Graph radiant 26000
Graph radiant 27000
Graph radiant 28000
Graph radiant 29000
Graph radiant 30000
Graph radiant 31000
Graph radiant 32000
Graph radiant 33000
Graph radiant 34000
Graph radiant 35000
Graph radiant 36000
Graph radiant 37000
Graph radiant 38000
Graph radiant 39000
Graph radiant 40000
Graph radiant 41000
Graph radiant 42000
Graph radiant 43000
Graph radiant 44000
Graph radiant 45000
Graph radiant 46000
Graph radiant 47000
Graph radiant 48000
Graph radiant 49000
Graph rad